In [ ]:
!pip install cohere umap-learn altair annoy datasets tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.0 MB/s eta 0:00:00
  Created wheel for cohere: filename=cohere-3.2.3-cp38-cp38-linux_x86_64.whl size=13276 sha2

In [ ]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [ ]:
api_key = '7I6LnJOim8wuqStD4jHvugh0k8bNTaImDVH2ri1e'
co = cohere.Client(api_key)

In [ ]:
`dataset = load_dataset("trec", split="train")

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


In [ ]:
dataset = load_dataset("trec", split="train")

In [ ]:
df = pd.DataFrame(dataset)[:1000]

In [ ]:
df.head(10)

,text,coarse_label,fine_label
0,How did serfdom develop in and then leave Russia ?,2,26
1,What films featured the character Popeye Doyle ?,1,5
2,How can I find a list of celebrities ' real names ?,2,26
3,What fowl grabs the spotlight after the Chinese Year of the Monkey ?,1,2
4,What is the full form of .com ?,0,1
5,What contemptible scoundrel stole the cork from my lunch ?,3,29
6,What team did baseball 's St. Louis Browns become ?,3,28
7,What is the oldest profession ?,3,30
8,What are liver enzymes ?,2,24
9,Name the scar-faced bounty hunter of The Old West .,3,29


In [ ]:
embeds = co.embed(texts=list(df['text']),
                  model="large",
                  truncate="RIGHT").embeddings

In [ ]:
embeds = np.array(embeds)
embeds.shape

(1000, 4096)

In [ ]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10)
search_index.save('test.ann')

True

In [ ]:
example_id = 92
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]}).drop(example_id)
print(f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:")
results

Question:'What are bear and bull markets ?'
Nearest neighbors:


,texts,distance
614,What animals do you find in the stock market ?,0.882656
137,What are equity securities ?,1.057621
307,What does NASDAQ stand for ?,1.077861
547,Where can stocks be traded on-line ?,1.090762
513,What do economists do ?,1.121739
363,What does it mean `` Rupee Depreciates '' ?,1.130840
922,What is the difference between a median and a mean ?,1.132366
601,What is `` the bear of beers '' ?,1.143285
932,Why did the world enter a global depression in 1929 ?,1.152497


In [ ]:
query = "What is the tallest mountain in the world?"
query_embed = co.embed(texts=[query],
                  model="large",
                  truncate="RIGHT").embeddings
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
results

Query:'What is the tallest mountain in the world?'
Nearest neighbors:


,texts,distance
236,What is the name of the tallest mountain in the world ?,0.483428
670,What is the highest mountain in the world ?,0.545002
412,What was the highest mountain on earth before Mount Everest was discovered ?,0.795815
435,What is the highest peak in Africa ?,0.917559
109,Where is the highest point in Japan ?,0.970060
907,What mountain range is traversed by the highest railroad in the world ?,0.985462
901,What 's the longest river in the world ?,1.002430
114,What is the largest snake in the world ?,1.036450
27,What is the highest waterfall in the United States ?,1.041542
354,What ocean is the largest in the world ?,1.060642


In [ ]:
reducer = umap.UMAP(n_neighbors=20)
umap_embeds = reducer.fit_transform(embeds)
df_explore = pd.DataFrame(data={'text': df['text']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)